### Import Librarires And Dataset

In [1]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#some settings to show data
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

target_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data")
# target_url = "datasets/abalone.data" # if you work local dataset, more fast than a link
abalone_df = pd.read_csv(target_url)
headers = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
abalone_df.columns = headers

### Analyze the Data

In [2]:
abalone_df.head()

In [3]:
abalone_df.info()

**Get target value**

In [4]:
# If you want the target values to be numeric rather than categorical, actually if you want to regression, you can comment this cell
for index in abalone_df.index:
    row = abalone_df.loc[index]
    if row["Rings"] <= 8:
        abalone_df.loc[index, 'Rings'] = 'Young'
    elif row["Rings"] >= 11:
        abalone_df.loc[index, 'Rings'] = 'Old'
    elif row["Rings"] >=9 & row["Rings"] <= 10:
        abalone_df.loc[index, 'Rings'] = 'Medium'

In [5]:
def is_numeric_value(x):
    return type(x) == int or type(x) == float

In [6]:
class MyQuestioner:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def __repr__(self):
        status = "=="
        if is_numeric_value(self.value):
            status = ">="
        return f"Is {headers[self.column]} {status} {self.value}"

    def compare(self, compared):
        val = compared[self.column]
        if is_numeric_value(val):
            return val >= self.value
        else:
            return val == self.value

In [7]:
def partitioner(rows, my_question):
    true_rows, false_rows = [], []
    for row in rows:
        if my_question.compare(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [8]:
def class_counts(rows):
    counts = {}  # a dictionary -> { label: count }
    for row in rows:
        label = row[-1]  # in our dataset format, the label is the last column, so -1
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [9]:
def gini_impurity(rows):  # There are some ways like entropy, but I use gini impurity :
    #  https://medium.com/machine-learning-t%C3%BCrkiye/karar-agaclari-algoritmasi-b823c23997d0#:~:text=Gini%20impurity%20nedir,%C3%B6l%C3%A7mek%20i%C3%A7in%20kullan%C4%B1labilir.

    impurity_value = 1
    counts = class_counts(rows)

    for label in counts:
        probability_of_label = counts[label] / float(len(rows))
        impurity_value -= probability_of_label ** 2

    return impurity_value

In [10]:
def information_gain(left, right, current_uncertainty):
    # Information Gain: The uncertainty of the starting node, minus the weighted impurity of two child nodes.

    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - (1 - p) * gini_impurity(right) - p * gini_impurity(left)

In [11]:
def find_the_best_split(rows):
    best_gain = 0  # best information gain
    best_question = None
    n_features = len(rows[0]) - 1  # column numbers
    current_uncertainty = gini_impurity(rows)

    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value
            question = MyQuestioner(col, val)
            true_rows, false_rows = partitioner(rows, question)  # try partitioner

            if len(true_rows) == 0 or len(false_rows) == 0:  # if there is no dataset, skip
                continue

            gain = information_gain(true_rows, false_rows, current_uncertainty)

            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [12]:
class Leaf:  # Leaf node is last of three, it has a value, has not any tree
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [13]:
class DecisionNode:  # A Decision Node has two branch true or false
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [14]:
 def predict_all(x, tree, options):
    preditions = []

    for row in x:
        cl = classify(row, tree)
        preditions.append(max(cl, key=cl.get))

    # also I can write above code like this; preditions = [max(classify(row, tree), key=classify(row, tree).get) for row in X]
    #   but for loop like above is more clear and understandable
    return preditions

In [15]:
def predict_one(counts):
    predicted_val = max(counts, key=counts.get)
    return predicted_val

In [16]:
def show_my_tree(node, space=""):
    if isinstance(node, Leaf):
        print(space + "Predict", predict_one(node.predictions)) # To write in line as a same indent
        return

    print(space + str(node.question)) # Ask question

    print(space + '--> False/Left:')
    show_my_tree(node.false_branch, space + "  ")

    print(space + '--> True/Right:')
    show_my_tree(node.true_branch, space + "  ")

In [17]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.question.compare(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [18]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [19]:
def builder_without_pruning(rows, attribute_types):
    gain, question = find_the_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partitioner(rows, question)
    true_branch = builder_without_pruning(true_rows, attribute_types)  # Recursive call without pruning
    false_branch = builder_without_pruning(false_rows, attribute_types)  # Recursive call without pruning

    return DecisionNode(question, true_branch, false_branch)

In [20]:
def builder_with_pruning(rows, max_depth, attribute_types, curr_depth=0):
    if curr_depth <= max_depth:
        gain, question = find_the_best_split(rows)

        if gain == 0:
            return Leaf(rows)

        true_rows, false_rows = partitioner(rows, question)
        true_branch = builder_with_pruning(true_rows, max_depth, attribute_types, curr_depth + 1)  # Recursive call
        false_branch = builder_with_pruning(false_rows, max_depth, attribute_types, curr_depth + 1)  # Recursive call

        return DecisionNode(question, true_branch, false_branch)

    return Leaf(rows)

In [21]:
def build_dt(X, y, attribute_types, options):
    rows = np.concatenate((X, y), axis=1).tolist()

    if options["pruning"]:
        return builder_with_pruning(rows, options["max_depth"], attribute_types)
    elif not options["pruning"]:
        return builder_without_pruning(rows, attribute_types)

In [22]:
def predict_dt(dt, X, options):
    return np.array(predict_all(X, dt, options))

In [23]:
X = abalone_df.iloc[:, :-1].values
y = abalone_df.iloc[:, -1].values.reshape(-1, 1)

options = {"max_depth": 5, "pruning": False}
attribute_types = abalone_df.dtypes.apply(str).tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)

### Show Tree

In [24]:
my_tree = build_dt(X_train, y_train, attribute_types, options)
y_pred = predict_dt(my_tree, X_test, options)
show_my_tree(my_tree)

### Find Confusion Matrices Without K-fold Cross Validation

In [25]:
print(f"Accuarcy score: {accuracy_score(y_test, y_pred)}")
confusion_matrix(y_test, y_pred)

### Find Confusion Matrices With K-fold Cross Validation

In [26]:
kf = KFold(n_splits=6, shuffle=True, random_state=42)
# Lists to store accuracy scores & confusion_matrices for each fold

def cross_val(options):
    # Perform k-fold cross-validation via kf.split(X_train), this function give me an indexes subset of X_train
    accuracy_scores = []
    confusion_matrices = []

    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        my_tree = build_dt(X_train_fold, y_train_fold, attribute_types, options)
        y_pred_fold = predict_dt(my_tree, X_val_fold, options)

        # Calculate accuracy and confusion_matrix for current fold
        accuracy = accuracy_score(y_val_fold, y_pred_fold)
        cm = confusion_matrix(y_val_fold, y_pred_fold)

        # Store accuracy score and confusion_matrix
        accuracy_scores.append(accuracy)
        confusion_matrices.append(cm)

    return accuracy_scores, confusion_matrices

In [27]:
def plot_confusion_matrix_accuracy_score(accuracy_scores, confusion_matrices):
    for i, score in enumerate(accuracy_scores):
        plt.figure(figsize=(3, 3))
        sns.heatmap(confusion_matrices[i], annot=True, fmt='d', cmap='Greens')
        plt.title('Confusion Matrix')
        plt.ylabel('True label')
        i+=1
        plt.xlabel(f'{i}. Fold cross validation Score: {score:.3f}')
        plt.plot()

    plt.show()

    # Calculate average accuracy across all folds
    avg_accuracy = np.mean(accuracy_scores)
    print(f"Average accuracy: {avg_accuracy}")

### Without pruning

In [28]:
accuracy_scores, confusion_matrices = cross_val(options)
plot_confusion_matrix_accuracy_score(accuracy_scores, confusion_matrices)

### With pruning

In [29]:
options["pruning"] = True
accuracy_scores, confusion_matrices = cross_val(options)
plot_confusion_matrix_accuracy_score(accuracy_scores, confusion_matrices)

### As you can see
*Without cross-validation without pruning
-> Accuarcy score:*  **~ 0.55**

*With cross-validation without pruning
-> Accuarcy score:*  **~ 0.57**

*With cross-validation with pruning
-> Accuarcy score:*  **~ 0.63**

### Let's Go Random Forest

In [30]:
from collections import Counter

def build_rdf(X, y, attribute_types, N, options):
    trees = []
    for _ in range(N):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        X_sample = X[idxs]
        y_sample = y[idxs]
        tree = build_dt(X_sample, y_sample, attribute_types, options)
        trees.append(tree)
    return trees



In [31]:
def predict_rdf(rdf, X, options):
    predictions = np.array([predict_dt(tree, X, options) for tree in rdf])
    tree_preds = np.swapaxes(predictions, 0, 1)
    predictions = np.array([most_frequently_label(pred) for pred in tree_preds])
    return predictions

def most_frequently_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [32]:
def cross_val_rdf(options, N):
    # Perform k-fold cross-validation via kf.split(X_train), this function give me an index subset of X_train
    accuracy_scores = []
    confusion_matrices = []

    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        rdf = build_rdf(X_train_fold, y_train_fold, attribute_types, N, options)
        y_pred_rdf_fold = predict_rdf(rdf, X_val_fold, options)

        # Calculate accuracy and confusion_matrix for current fold
        accuracy = accuracy_score(y_val_fold, y_pred_rdf_fold)
        cm = confusion_matrix(y_val_fold, y_pred_rdf_fold)

        # Store accuracy score and confusion_matrix
        accuracy_scores.append(accuracy)
        confusion_matrices.append(cm)

    return accuracy_scores, confusion_matrices

In [33]:
accuracy_scores, confusion_matrices = cross_val_rdf(options, 2)
plot_confusion_matrix_accuracy_score(accuracy_scores, confusion_matrices)

### As you can see
*With Random Forest and N = 2
-> Accuarcy score:*  **~ 0.60**

Actually, the score is too good, because I set only N = 2 for a performance problem, even though this operation took approximately 70 seconds. So if you test N = 5 or N = 10 paramater, it will take larger processing time between 500 or 1000 seconds.